## Jieba Demo

* 基于Python的中文分词工具
* https://github.com/fxsjy/jieba

#### 特点

支持三种分词模式：

1. **精确模式**，试图将句子最精确地切开，适合文本分析；
2. **全模式**，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义；
3. **搜索引擎模式**，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。
4. 支持繁体分词
5. 支持自定义词典
6. MIT 授权协议

* jieba.cut ： 方法接受三个输入参数: 需要分词的字符串；cut_all 参数用来控制是否采用全模式；HMM 参数用来控制是否使用 HMM 模型

* jieba.cut_for_search ： 方法接受两个参数：需要分词的字符串；是否使用 HMM 模型。该方法适合用于搜索引擎构建倒排索引的分词，粒度比较细待分词的字符串可以是 unicode 或 UTF-8 字符串、GBK 字符串。注意：不建议直接输入 GBK 字符串，可能无法预料地错误解码成 UTF-8

> jieba.cut 以及 jieba.cut_for_search 返回的结构都是一个可迭代的 generator，可以使用 for 循环来获得分词后得到的每一个词语(unicode)，或者用jieba.lcut 以及jieba.lcut_for_search 直接返回 list

* jieba.Tokenizer(dictionary=DEFAULT_DICT) ： 新建自定义分词器，可用于同时使用不同词典。jieba.dt 为默认分词器，所有全局分词相关函数都是该分词器的映射。

In [1]:
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

'''
【全模式】: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学

【精确模式】: 我/ 来到/ 北京/ 清华大学

【新词识别】：他, 来到, 了, 网易, 杭研, 大厦    (此处，“杭研”并没有在词典中，但是也被Viterbi算法识别出来了)

【搜索引擎模式】： 小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造
'''

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/np/8_1r_3md1l755l1wrlx_q0sc0000gn/T/jieba.cache
Loading model cost 0.975 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


'\n【全模式】: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学\n\n【精确模式】: 我/ 来到/ 北京/ 清华大学\n\n【新词识别】：他, 来到, 了, 网易, 杭研, 大厦    (此处，“杭研”并没有在词典中，但是也被Viterbi算法识别出来了)\n\n【搜索引擎模式】： 小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造\n'

### 载入词典

开发者可以指定自己自定义的词典，以便包含 jieba 词库里没有的词。虽然 jieba 有新词识别能力，但是自行添加新词可以保证更高的正确率

用法： jieba.load_userdict(file_name) # file_name 为文件类对象或自定义词典的路径
> 词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。

词频省略时使用自动计算的能保证分出该词的词频。

In [2]:
# 用户添加自定词库 jieba.load_userdict(file_name)
'''
词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：
词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。
file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
'''
str_demo = '李小福是创新办主任也是云计算方面的专家'
jieba.load_userdict('user_dict_jieba.txt')
seg_user = jieba.cut(str_demo)
print('|'.join(seg_user))

李小福|是|创新办|主任|也|是|云计算|方面|的|专家


### 调整词典

* 使用 add_word(word, freq=None, tag=None) 和 del_word(word) 可在程序中动态修改词典。

* 使用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。

**注意**：自动计算的词频在使用 HMM 新词发现功能时可能无效。

In [3]:
# 调整词典
'''
使用 add_word(word, freq=None, tag=None) 和 del_word(word) 可在程序中动态修改词典。
使用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。
注意：自动计算的词频在使用 HMM 新词发现功能时可能无效。
'''

test_str = '如果放到post中将出错。'
seg1 = jieba.cut(test_str, HMM=False)
print('seg1: ' + '|'.join(seg1))
# 改变词频
jieba.suggest_freq(('中', '将'), True)
seg2 = jieba.cut(test_str, HMM=False)
print('seg2: ' + '|'.join(seg2))

test_str2 = '「台中」正确应该不会被切开'
jieba.suggest_freq('台中', True)
print('seg3: ' + '|'.join(jieba.cut(test_str2, HMM=False)))
jieba.suggest_freq(('台', '中'), True)
print('seg4: ' + '|'.join(jieba.cut(test_str2, HMM=False)))

seg1: 如果|放到|post|中将|出错|。
seg2: 如果|放到|post|中|将|出错|。
seg3: 「|台中|」|正确|应该|不会|被|切开
seg4: 「|台|中|」|正确|应该|不会|被|切开


## 关键词提取

### 基于 TF-IDF 算法的关键词抽取
```
import jieba.analyse

jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
sentence 为待提取的文本
topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
withWeight 为是否一并返回关键词权重值，默认值为 False
allowPOS 仅包括指定词性的词，默认值为空，即不筛选
jieba.analyse.TFIDF(idf_path=None) 新建 TFIDF 实例，idf_path 为 IDF 频率文件
```

[代码示例 （关键词提取）](https://github.com/fxsjy/jieba/blob/master/test/extract_tags.py)

* 关键词提取所使用逆向文件频率（IDF）文本语料库可以切换成自定义语料库的路径

用法： jieba.analyse.set_idf_path(file_name) # file_name为自定义语料库的路径

[自定义语料库示例：](https://github.com/fxsjy/jieba/blob/master/extra_dict/idf.txt.big)

[用法示例：](https://github.com/fxsjy/jieba/blob/master/test/extract_tags_idfpath.py)


* 关键词提取所使用停止词（Stop Words）文本语料库可以切换成自定义语料库的路径

用法： jieba.analyse.set_stop_words(file_name) # file_name为自定义语料库的路径

[自定义语料库示例：](https://github.com/fxsjy/jieba/blob/master/extra_dict/stop_words.txt)

[用法示例：](https://github.com/fxsjy/jieba/blob/master/test/extract_tags_stop_words.py)

* 关键词一并返回关键词权重值示例

[用法示例：](https://github.com/fxsjy/jieba/blob/master/test/extract_tags_with_weight.py)

In [4]:
# 基于TF-IDF算法的关键词抽取

import jieba
import jieba.analyse

sentence = '全国港澳研究会会长徐泽在会上发言指出，学习系列重要讲话要深刻领会 主席关于香港回归后的宪制基础和宪制秩序的论述，这是过去20年特别是中共十八大以来"一国两制"在香港实践取得成功的根本经验。首先，要在夯实 香港的宪制基础、巩固香港的宪制秩序上着力。只有牢牢确立起"一国两制"的宪制秩序，才能保证"一国两制"实践不走样 、不变形。其次，要在完善基本法实施的制度和机制上用功。中央直接行使的权力和特区高度自治权的结合是特区宪制秩 序不可或缺的两个方面，同时必须切实建立以行政长官为核心的行政主导体制。第三，要切实加强香港社会特别是针对公 职人员和青少年的宪法、基本法宣传，牢固树立"一国"意识，坚守"一国"原则。第四，要努力在全社会形成聚焦发展、抵 制泛政治化的氛围和势能，全面准确理解和落实基本法有关经济事务的规定，使香港继续在国家发展中发挥独特作用并由 此让最广大民众获得更实在的利益。'

keywords = jieba.analyse.extract_tags(sentence, topK=10, withWeight=True, allowPOS=('n','nr','ns'))

for item in keywords:
    print(item)
#     print(item[0],item[1])

('宪制', 1.3431386920067796)
('基本法', 0.5232081223372882)
('香港', 0.4925521903983051)
('秩序', 0.37291551977135595)
('特区', 0.307537759819322)
('香港回归', 0.21210819137118644)
('徐泽', 0.20262317801525423)
('政治化', 0.1875841408118644)
('走样', 0.18569896055593219)
('势能', 0.1692992312974576)


### 基于 TextRank 算法的关键词抽取

jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。

jieba.analyse.TextRank() 新建自定义 TextRank 实例。


In [5]:
import jieba

sentence = '全国港澳研究会会长徐泽在会上发言指出，学习系列重要讲话要深刻领会 主席关于香港回归后的宪制基础和宪制秩序的论述，这是过去20年特别是中共十八大以来"一国两制"在香港实践取得成功的根本经验。首先，要在夯实 香港的宪制基础、巩固香港的宪制秩序上着力。只有牢牢确立起"一国两制"的宪制秩序，才能保证"一国两制"实践不走样 、不变形。其次，要在完善基本法实施的制度和机制上用功。中央直接行使的权力和特区高度自治权的结合是特区宪制秩 序不可或缺的两个方面，同时必须切实建立以行政长官为核心的行政主导体制。第三，要切实加强香港社会特别是针对公 职人员和青少年的宪法、基本法宣传，牢固树立"一国"意识，坚守"一国"原则。第四，要努力在全社会形成聚焦发展、抵 制泛政治化的氛围和势能，全面准确理解和落实基本法有关经济事务的规定，使香港继续在国家发展中发挥独特作用并由 此让最广大民众获得更实在的利益。'

for x, w in jieba.analyse.textrank(sentence, withWeight=True):
    print('%s %s' % (x, w))

宪制 1.0
香港 0.6752032702819407
基本法 0.6089395571311746
秩序 0.5152715259580873
发展 0.4463006469921722
基础 0.3642842474536471
核心 0.3561574604755039
发言 0.3196934303211121
会长 0.31703386747268975
实践 0.30901531844265184
获得 0.3038805346070096
实在 0.3016932834659774
制度 0.3006210302608225
势能 0.2994079945089196
有关 0.2924969444979404
特区 0.2809975020048608
规定 0.2758357239811501
行政长官 0.2728163219688171
行政 0.2717758063343893
理解 0.2675120620331688


### 词性标注

*  jieba.posseg.POSTokenizer(tokenizer=None) 新建自定义分词器，tokenizer 参数可指定内部使用的 jieba.Tokenizer 分词器。jieba.posseg.dt 为默认词性标注分词器。

* 标注句子分词后每个词的词性，采用和 ictclas 兼容的标记法。

In [6]:
# 词性标注
import jieba.posseg as pseg

words = pseg.cut("我爱北京天安门")
for word, flag in words:
    print('%s %s' % (word, flag))


我 r
爱 v
北京 ns
天安门 ns


### 并行分词

* jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数
* jieba.disable_parallel() # 关闭并行分词模式

In [7]:
import sys
import time
sys.path.append("../../")
import jieba

import requests

# 开启 并行分词
jieba.enable_parallel()
# 关闭 并行分词
# jieba.disable_parallel()

url = 'http://36kr.com/p/5068410.html'
req = requests.get(url)
content = req.text
# print(req.content)

# req = urllib.request.Request(url)
# req.add_header('User-Agent', 'Mozilla/5.0')
# content = urllib.request.urlopen(req).read()

t1 = time.time()
words = "/ ".join(jieba.cut(content))

t2 = time.time()
tm_cost = t2-t1

# log_f = open("1.log","wb")
# log_f.write(words.encode('utf-8'))

print('speed %s bytes/second' % (len(content)/tm_cost))

speed 498821.18822654383 bytes/second


In [8]:
# Tokenize：返回词语在原文的起止位置
import jieba

result = jieba.tokenize('永和服装饰品有限公司')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

print('='*40)
print('搜索模式')
print('='*40)
    
result_search = jieba.tokenize('永和服装饰品有限公司', mode='search')
for tk in result_search:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限公司		 start: 6 		 end:10
搜索模式
word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限		 start: 6 		 end:8
word 公司		 start: 8 		 end:10
word 有限公司		 start: 6 		 end:10


In [9]:
import sys
sys.path.append("../")

import jieba
import jieba.posseg
import jieba.analyse

print('='*40)
print('1. 分词')
print('-'*40)

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 默认模式

seg_list = jieba.cut("他来到了网易杭研大厦")
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

print('='*40)
print('2. 添加自定义词典/调整词典')
print('-'*40)

print('/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))
#如果/放到/post/中将/出错/。
print(jieba.suggest_freq(('中', '将'), True))
#494
print('/'.join(jieba.cut('如果放到post中将出错。', HMM=False)))
#如果/放到/post/中/将/出错/。
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))
#「/台/中/」/正确/应该/不会/被/切开
print(jieba.suggest_freq('台中', True))
#69
print('/'.join(jieba.cut('「台中」正确应该不会被切开', HMM=False)))
#「/台中/」/正确/应该/不会/被/切开

print('='*40)
print('3. 关键词提取')
print('-'*40)
print(' TF-IDF')
print('-'*40)

s = "此外，公司拟对全资子公司吉林欧亚置业有限公司增资4.3亿元，增资后，吉林欧亚置业注册资本由7000万元增加到5亿元。吉林欧亚置业主要经营范围为房地产开发及百货零售等业务。目前在建吉林欧亚城市商业综合体项目。2013年，实现营业收入0万元，实现净利润-139.13万元。"
for x, w in jieba.analyse.extract_tags(s, withWeight=True):
    print('%s %s' % (x, w))

print('-'*40)
print(' TextRank')
print('-'*40)

for x, w in jieba.analyse.textrank(s, withWeight=True):
    print('%s %s' % (x, w))

print('='*40)
print('4. 词性标注')
print('-'*40)

words = jieba.posseg.cut("我爱北京天安门")
for word, flag in words:
    print('%s %s' % (word, flag))

print('='*40)
print('6. Tokenize: 返回词语在原文的起止位置')
print('-'*40)
print(' 默认模式')
print('-'*40)

result = jieba.tokenize('永和服装饰品有限公司')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

print('-'*40)
print(' 搜索模式')
print('-'*40)

result = jieba.tokenize('永和服装饰品有限公司', mode='search')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

1. 分词
----------------------------------------
Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造
2. 添加自定义词典/调整词典
----------------------------------------
如果/放到/post/中/将/出错/。
494
如果/放到/post/中/将/出错/。
「/台/中/」/正确/应该/不会/被/切开
69
「/台/中/」/正确/应该/不会/被/切开
3. 关键词提取
----------------------------------------
 TF-IDF
----------------------------------------
欧亚 0.7300142700289363
吉林 0.659038184373617
置业 0.4887134522112766
万元 0.3392722481859574
增资 0.33582401985234045
4.3 0.25435675538085106
139.13 0.25435675538085106
7000 0.25435675538085106
2013 0.25435675538085106
实现 0.19900979900382978
综合体 0.19480309624702127
经营范围 0.19389757253595744
亿元 0.1914421623587234
在建 0.17541884768425534
全资 0.17180164988510638
注册资本 0.1712441526
百货 0.16734460041382979
零售 0.1475057117057447
子公司 0.14596045237787234
营业 0.13920178509021275
----------------------------------------
 TextRank
------------------------